In [1]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from PIL import Image
import numpy as np
import os

Using TensorFlow backend.
C:\Users\saira\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\saira\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\saira\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\saira\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [3]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [4]:
# load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [5]:
# Reshaping to format which CNN expects (batch, height, width, channels)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1).astype('float32')

In [6]:
# To load images to features and labels
def load_images_to_data(image_label, image_directory, features_data, label_data):
    list_of_files = os.listdir(image_directory)
    for file in list_of_files:
        image_file_name = os.path.join(image_directory, file)
        if ".png" in image_file_name:
            img = Image.open(image_file_name).convert("L")
            img = np.resize(img, (28,28,1))
            im2arr = np.array(img)
            im2arr = im2arr.reshape(1,28,28,1)
            features_data = np.append(features_data, im2arr, axis=0)
            label_data = np.append(label_data, [image_label], axis=0)
    return features_data, label_data

In [7]:
# Load your own images to training and test data
X_train, y_train = load_images_to_data('1', 'data/mnist_data/train/1', X_train, y_train)
X_test, y_test = load_images_to_data('1', 'data/mnist_data/validation/1', X_test, y_test)

In [8]:
# normalize inputs from 0-255 to 0-1
X_train/=255
X_test/=255

In [9]:
# one hot encode
number_of_classes = 10
y_train = np_utils.to_categorical(y_train, number_of_classes)
y_test = np_utils.to_categorical(y_test, number_of_classes)

In [10]:
# create model
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(X_train.shape[1], X_train.shape[2], 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(number_of_classes, activation='softmax'))

In [11]:
# Compile model
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [12]:
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=7, batch_size=200)


Train on 60005 samples, validate on 10002 samples
Epoch 1/7
60005/60005 [==============================] - 43s 711us/step - loss: 0.4906 - accuracy: 0.8420 - val_loss: 0.0815 - val_accuracy: 0.9749
Epoch 2/7
60005/60005 [==============================] - 38s 640us/step - loss: 0.1592 - accuracy: 0.9524 - val_loss: 0.0568 - val_accuracy: 0.9824
Epoch 3/7
60005/60005 [==============================] - 39s 642us/step - loss: 0.1207 - accuracy: 0.9630 - val_loss: 0.0478 - val_accuracy: 0.9851
Epoch 4/7
60005/60005 [==============================] - 39s 648us/step - loss: 0.1000 - accuracy: 0.9696 - val_loss: 0.0396 - val_accuracy: 0.9875
Epoch 5/7
60005/60005 [==============================] - 39s 645us/step - loss: 0.0861 - accuracy: 0.9741 - val_loss: 0.0335 - val_accuracy: 0.9892
Epoch 6/7
60005/60005 [==============================] - 39s 643us/step - loss: 0.0801 - accuracy: 0.9755 - val_loss: 0.0316 - val_accuracy: 0.9901
Epoch 7/7
60005/60005 [==============================] - 39s 

In [13]:
# Save the model
# model.save('models/mnistCNN.h5')

In [14]:
# Final evaluation of the model
metrics = model.evaluate(X_test, y_test, verbose=0)
print("Metrics(Test loss & Test Accuracy): ")
print(metrics)

Metrics(Test loss & Test Accuracy): 
[0.029722471712944638, 0.9894021153450012]


In [15]:
img = Image.open('data/mnist_data/validation/1/1_2.png').convert("L")
img = np.resize(img, (28,28,1))
im2arr = np.array(img)
im2arr = im2arr.reshape(1,28,28,1)

In [16]:
y_pred = model.predict_classes(im2arr)
print(y_pred)

[1]
